# RNN (循环神经网络)

以前我们写过的 Network 的结果都是反馈给下一层。

那在RNN中，我们不仅要输出结果，还要输出一个**隐含状态**，给当前层在处理下一个样本时使用。

结构图如下所示：

![Recurrent Neural Networks have loops](../images/RNN/RNN-rolled.jpg)


可以看到，跟普通的 Neural Network 相比，我们这里仅仅是多了一个**隐含状态**，那么这个多出来的**隐含状态**有什么好处呢？

这个**隐含状态**，我们可以看作是上一个样本遗留给我们的信息，通过这个信息，我们可以把握过去样本的特征，结合当前的样本，组成一种类似于 “时间序列” 样本信息，从而让模型有了处理前后有依赖关系数据样本的能力。

**举个例子:**

语言模型的任务是给定句子的前 $t$ 个字符，然后预测第 $t+1$ 个字符。假设我们的句子是“你好世界”，使用循环神经网络来预测的一个做法是，在时间 $1$ 输入“你”，预测“好”同时生成一个**隐含状态**；

在时间 $2$ 向同一个网络输入刚才生成的**隐含状态** 和 “好” 去预测“世”。同时输出一个更新过的隐含状态。

对于这个**隐含状态**， 我们会希望前面的信息能够保存在这个隐含状态里，从而提升预测效果。下图展示了这个过程。

![Recurrent Neural Networks have loops](../images/RNN/RNN.jpg)




# RNN的公式表示

针对输入输出加上时间状态 $t$。在 $t$ 时刻，我们的输入为 $X_t$ 和 $H_{t-1}$，经过一个隐含层之后，我们有输出 $H_t$，经过最后的输出层得到 $\hat{Y_t}$。

其中 $X_t \in R^{n \times x}$ (样本数量为 $n$，每个样本特征向量维度是 $x$) ； $H_{t-1} \in R^{n \times h}$ （其中隐含层长度为 $h$）

隐含层的计算公式为：

$$\mathbf{H}_t = \phi(\mathbf{X}_t \mathbf{W}_{xh} + \mathbf{H}_{t-1} \mathbf{W}_{hh}  + \mathbf{b}_h)$$

其中 $\mathbf{W}_{xh}$ 和 $\mathbf{W}_{hh}$ 是隐含层的权重， $\mathbf{b}_h$ 是隐含层的 bias。

输出层的计算公式为：

$$\hat{\mathbf{Y}}_t = \text{softmax}(\mathbf{H}_t \mathbf{W}_{hy}  + \mathbf{b}_y)$$

其中 $\mathbf{W}_{hy}$ 是输出层权重，$\mathbf{b}_y$ 是输出层的 bias


最开始的隐含状态里的元素通常会被初始化为0，也就是 $H_0$ 是个零矩阵。






循环神经网络的简单介绍就到这里了，如果想要更加细致的了解，可以看本人写过的一篇Blog [详细剖析RNN理论、变种和应用](https://lianhaimiao.github.io/2018/02/09/%E8%AF%A6%E7%BB%86%E5%89%96%E6%9E%90RNN%E7%90%86%E8%AE%BA%E3%80%81%E5%8F%98%E7%A7%8D%E5%92%8C%E5%BA%94%E7%94%A8/#more)

# RNN的 PyTorch 实现

理论部分我们已经了解了，那么下实际操作中，我们如何用 PyTorch 写一个RNN呢？


答案很简单，就是一个函数...

```
torch.nn.RNN()
```

对的，你没看错，实现 RNN 就是这么简单。

在 pyTroch 中你实现 RNN 模型的时候，可以传入的参数有：

> input_size – The number of expected features in the input x

> hidden_size – The number of features in the hidden state h

> num_layers – Number of recurrent layers.

> nonlinearity – The non-linearity to use [‘tanh’|’relu’]. Default: ‘tanh’

> bias – If False, then the layer does not use bias weights b_ih and b_hh. Default: True

> batch_first – If True, then the input and output tensors are provided as (batch, seq, feature)

> dropout – If non-zero, introduces a dropout layer on the outputs of each RNN layer except the last layer

> bidirectional – If True, becomes a bidirectional RNN. Default: False


在这里我们需要重点关注的是 **input_size**、**hidden_size**、**batch_first**

这三个分别表示，输入数据的特征大小、隐含层特征大小、batch放在第一位。

模型实现之后，就是输入数据的格式了。如果在模型实现的时候选择了**batch_first=True**，那么在输入数据的格式就会是 (batch, seq_len, input_size)


OK，整个RNN模型的 PyTorch 的理论部分，先介绍到这里，接下来的练习是**使用 RNN 模型来生成恐龙名字**


# 使用 RNN 模型来生成恐龙名字

**PS：该练习的原始数据和想法来自于 吴恩达 Deep Learning 系列课程第五课**

任务描述：

现在，我们有一堆恐龙的名字，我们希望通过训练一个 RNN 模型，在我们给定一个字符的前提下，模型可以自动为我们生成一个酷炫的恐龙名字。




为了实现这个任务，我们一共分六步走：

第一步：字符的数值表示 和 One-Hot Encoding

第二步：构造训练集

第三步：构造模型

第四步：构建更新梯度和梯度剪裁的函数

第五步：随机采样

第六步：准备工作完成，开始训练


其中，第四步，如果不愿意构建梯度裁剪函数，可以直接使用 torch.nn.optim 中提供的优化器来对模型进行更新。

最后，在PyTorch中损失函数的定义一定要慎重！ 如果**损失函数是 NLLLoss(negative log likelihood loss)**模型的最后一层就需要使用 softmax，但是如果损失函数是CrossEntropyLoss，那么pytorch会自动给你加上Softmax()



## 第一步：字符的数值表示 和 One-Hot Encoding

因为字符没办法输入到模型里面，所以我们得先把数据里面所有不同的字符拿出来做成一个字典，然后可以把每个字符转成从0开始的索引(index)来方便之后的使用。

需要注意的是我们一共有 27 个字符，出了小写字母 a-z(26个) 之外，我们还会有一个字符就是 "\n" 这个字符在这里的含义是 < EOS \> (or "End of sentence") 



In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.autograd import Variable
import numpy as np
import random

In [2]:
# 构建 config 类来 控制所有参数
class Config(object):
    def __init__(self):
        self.path = '../dataSet/dinos.txt'
        self.data_size = None
        self.vocab_size = None
        self.char_to_ix = None
        self.ix_to_char = None
        self.output_size = None
        self.epoch = 1000
        self.lr = 0.05
        self.hidden_size = 50
        self.batch_size = 1
        self.maxValue = 10 # 防止梯度爆炸所使用
    
config = Config()

In [3]:
# 读取数据
data = open(config.path, 'r').read()
# 字符全部小写
data = data.lower()
# 查看有多少种字符
chars = list(set(data))
# 27个字符，所以输出也是 27
config.data_size, config.vocab_size, config.output_size = len(data), len(chars), len(chars)

print('数据集中一共有 %d 个字符，不同的字符一共有 %d 个。' % (config.data_size, config.vocab_size))

数据集中一共有 19909 个字符，不同的字符一共有 27 个。


In [4]:
config.char_to_ix = { ch:i for i,ch in enumerate(sorted(chars)) } # 字符到索引的映射
config.ix_to_char = { i:ch for i,ch in enumerate(sorted(chars)) } # 索引到字符的映射
print(config.ix_to_char)

{0: '\n', 1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z'}


现在，我们已经得到了 字符转数字 和 数字转字符 的映射了，但是，这个现在的每个字符依旧是用一个整数表示的，而输入进网络我们需要一个定长的向量。一个常用的办法是使用 **One-Hot Encoding** 来将其表示成向量。也就是说，如果一个字符的整数值是 $i$ , 那么我们创建一个全0的长为 vocab_size 的向量，并将其第 $i$ 位设成1。该向量就是对原字符的 **one-hot 向量**。

In [5]:
def one_hot(ids, vocab_size):
    """
        ids: list
    """
    ids = torch.LongTensor(ids).view(-1, 1)
    out = torch.zeros(ids.size()[0], vocab_size).scatter_(1, ids, 1)
    return out

## 第二步：构造训练集

> 由于训练数据集中的数据长度不一致，所以，我们每次只训练一个样本。 当然有别的办法可以解决，但是在这个例子中，我们决定采用每次训练一个数据来训练模型

In [6]:
def train_dataset(path, char_to_ix, vocab_size):
    with open(path, 'r') as f:
        examples = f.readlines()
    examples = [x.lower().strip() for x in examples]
    for index in range(len(examples)):
        X = [char_to_ix[ch] for ch in examples[index]] 
        Y = X[1:] + [char_to_ix["\n"]]
        i_d = one_hot(X, vocab_size)
        input_data = Variable(i_d.view(1, i_d.size()[0], i_d.size()[1])) # batch*seq_len*input_size
        target_data = Variable(torch.LongTensor(Y))
        yield input_data, target_data

## 第三步：构造模型

我们使用最原始的RNN模型，构造的具体形式，如下图所示：

![Dinasor RNN Model](../images/RNN/rnn_model.png)


In [7]:
# 构造模型
class DinosaurModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, batch_size, num_layers=1):
        super(DinosaurModel, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.input_size = input_size
        self.batch_size = batch_size
        self.num_layers = num_layers
        # 就是一行就构建了 RNN 模型
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        
        self.fc  = nn.Linear(hidden_size, output_size)
    
    def forward(self, x, h0):
        # Propagate input through RNN
        # Input: (batch, seq_len, input_size)
        # hidden: (batch, num_layers * num_directions, hidden_size)
        # out: (batch, seq_len, hidden_size)
        out, hidden = self.rnn(x, h0)
        
        # 把 rnn 的结果堆成 (batch*seq_len, hidden_size) 大小
        out = out.view(out.size()[0]*out.size()[1], self.hidden_size)
        y = self.fc(out)
        return y
    
    def init_hidden(self):
        # 初始化隐含层
        return Variable(torch.zeros(self.batch_size, self.num_layers, self.hidden_size))

## 第四步：构建更新梯度和梯度剪裁的函数

在这里，我们不选择使用优化器，为了能够实现梯度裁剪，我们使用最原始的梯度下降法，同时手动实现 更新梯度的过程。

我们在训练循环神经网络的时候，当每个时序训练样本的时序长度 $T_x$ 较大或者时刻 $t$ 较小，那么目标函数有关 $t$ 时刻的隐含层变量梯度容易出现梯度消失(valishing) 或 梯度爆炸(explosion)


In [8]:
def clip_and_update(parameters, lr, maxValue):
    for p in parameters:
        gradients = torch.clamp(p.grad.data, min=-maxValue, max=maxValue)
        p.data.add_(-lr, gradients)
    return

## 第五步：随机采样

现在，假设我们的模型已经训练好了，我们需要生成新的文本（恐龙名字），生成的过程如下图所示：

![Sampling](../images/RNN/sampling.png)


我们把 $x^{\langle 1 \rangle} = \vec{0}$ 当作第一步的输入，然后让该网络每一步生成一个字符，一直到我们生成了结尾符 < EOS > 在这里的结尾符就是 "\n"


In [ ]:
def sample(model, char_to_ix, ix_to_char, vocab_size):
    # 初始值
    random_int = random.randint(0, vocab_size)
    i_d = one_hot([random_int], vocab_size)
    a_input = Variable(i_d).view(1, 1, vocab_size)
    indices = []
    idx = -1
    counter = 0
    eos = char_to_ix['\n']
    
    while (idx != eos and counter != 30):
        h0 = model.init_hidden()
        out = model(a_input, h0)
        # 通过 softmax 求出每个字符的概率
        p = F.softmax(out)
        # 取出概率最大的字符的位置
        val, ids = torch.max(p, 1)
        # 加入预测结果数组里面
        idx = ids.data[0]
        indices.append(idx)
        a_input = Variable(one_hot([idx], vocab_size).view(1, 1, vocab_size))
        counter += 1
    if (counter == 30):
        indices.append(char_to_ix['\n'])
    strl = [ix_to_char[i] for i in indices]
    return "".join(strl)

## 第六步：准备工作完成，开始训练

PS: 由于主要是讲述RNN，所以并没有特别认真的去训练，又兴趣的朋友可以试试 加上GPU， 同时多加训练次数，来尝试获取更好的结果


In [ ]:
# 定义损失函数: 因为最后一层用的是 softmax， 所以，这里我们使用 Negative log-likelihood function
dinos = DinosaurModel(config.vocab_size, config.hidden_size, config.output_size, config.batch_size)

loss_fn = nn.CrossEntropyLoss()

for i in range(config.epoch):    
    # 数据准备
    total_loss = []
    for input_data, target_data in train_dataset(config.path, config.char_to_ix, config.vocab_size):
        dinos.zero_grad()
        # 隐含层初始化
        h0 = dinos.init_hidden()
        # 数据扔进模型里面
        y_pred = dinos(input_data, h0)
        # 求 loss
        loss = loss_fn(y_pred, target_data)
        total_loss.append(loss.data[0])
        # 反向传播
        loss.backward()
        # 梯度剪裁同时更新模型
        clip_and_update(dinos.parameters(), config.lr, config.maxValue)

    if (i+1) % 20 == 0:
        print("loss is %.6f" %(np.mean(total_loss)))
        # 采样
        strl = sample(dinos, config.char_to_ix, config.ix_to_char, config.vocab_size)
        print(strl)

loss is 1.689902
uraurauraurauraurauraurauraura

loss is 1.560815
oraurauraurauraurauraurauraura

loss is 1.484745
oranoranoranoranoranoranoranor

loss is 1.429438
uranananananananananananananan

